# 유튜브 데이터 카테고리 분류

본 프로젝트는 전처리된 유튜브 데이터 샘플을 사용하여 **카테고리 분류**를 수행하는 과정을 다룹니다. 텍스트 데이터를 임베딩한 후 각 카테고리와 관련된 단어를 임베딩하여, 유사성을 기반으로 분류 작업을 수행합니다.

## 분류 알고리즘 개요
카테고리 분류는 **임베딩 기반 분류(Embedding-based Classification)** 또는 **거리 기반 분류(Distance-based Classification)** 기법을 사용하여 이루어집니다. 특히, **K-최근접 이웃(K-Nearest Neighbor)**과 같은 방법을 사용해 임베딩 벡터 간의 거리를 측정하여 분류 작업을 진행합니다.

임베딩 벡터 간의 **코사인 유사성(Cosine Similarity)**을 계산하여, 가장 유사한 카테고리를 찾는 방식으로 작동하며, 이를 통해 유튜브 비디오가 어느 카테고리에 속하는지 자동으로 예측합니다.

## 프로젝트 목표
이 프로젝트는 주로 **한국어 유튜브 채널**을 대상으로 하며, 국내 채널 마케팅을 위한 데이터 분석에 초점을 맞추고 있습니다. 따라서, 한국어로 이루어진 텍스트 데이터를 활용하여 카테고리를 분류하는 데 중점을 둡니다.

## 주요 단계:
1. **텍스트 임베딩**: 각 유튜브 비디오의 텍스트 데이터를 임베딩하여 벡터 공간에 매핑.
2. **카테고리 임베딩**: 카테고리와 관련된 단어들을 임베딩하여 해당 벡터를 생성.
3. **코사인 유사성 계산**: 비디오 임베딩과 카테고리 임베딩 간의 코사인 유사성을 측정하여 가장 유사한 카테고리를 선정.
4. **결과 분석**: 분류 결과를 분석하여 유튜브 비디오가 속할 가능성이 높은 카테고리를 확인.

이 과정은 마케팅 전략 수립에 필요한 인사이트를 제공하고, 채널의 주요 카테고리를 자동으로 분류하여 데이터 기반의 결정을 지원합니다.


In [9]:
import pandas as pd
import torch
import re
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoTokenizer, AutoModel

In [2]:
#### 카테고리 파일 불러오기
# matching_type: 데이터 분류 방법을 나타냅니다. (1 = 검색키워드 일치 기반 분류, 2 = 임베딩을 통한 카테고리 분류)
# 대분류: 유튜브 카테고리의 대분류를 나타냅니다. (예: 음악, 게임 등)
# 소분류: 대분류 하위의 세부 카테고리입니다. (예: 트로트, 노래 등)
# 검색키워드: 해당 카테고리 및 소분류와 관련된 주요 키워드 목록입니다. 키워드는 "/"로 구분됩니다.
# 업데이트: 데이터가 마지막으로 업데이트된 날짜를 나타냅니다.
# cluster: 각 카테고리 및 소분류에 할당된 클러스터 번호입니다.
# new_cluster: 새로운 클러스터 번호 (미정인 경우 NaN).
# category: 대분류와 소분류를 결합한 최종 카테고리 이름입니다. (예: 음악 트로트)

category_file = pd.read_csv("../data/youtube_video_category.csv", encoding="utf-8-sig", index_col=0)
len(category_file)
# ", ".join(category_file.category.tolist())
category_file

,matching_type,대분류,소분류,검색키워드,업데이트,cluster,new_cluster,category
0,2,음악,트로트,트로트/미스트롯/미스터트롯/트롯,2024-07-30,14,NaN,음악 트로트
1,1,음악,트로트,미스트롯/미스터트롯/트롯/임영웅/현역가왕/황영웅/TROT/현철/나훈아,2024-07-30,14,NaN,음악 트로트
2,2,음악,노래,노래/보컬/발성/메들리/노래방/플레이리스트/팝송/커버/노래모음/케이팝/신곡/재즈,2024-07-30,15,NaN,음악 노래
3,1,음악,노래,KPOP/kpop/Kpop/K-POP/PLAYLIST/COVER/OST/LYRICS...,2024-07-30,15,NaN,음악 노래
4,2,음악,댄스,댄스/춤/치어리더/직캠/무용,2024-07-30,16,NaN,음악 댄스
...,...,...,...,...,...,...,...,...
128,1,게임,게임,월드오브탱크/바둑,2024-08-08,5,NaN,게임 게임
129,2,스포츠,스포츠,탁구,2024-08-08,72,NaN,스포츠 스포츠
130,2,스포츠,스포츠,배드민턴/혼합복식/남자복식/여자복식,2024-08-08,72,NaN,스포츠 스포츠
131,1,금융,비트코인,코인,2024-08-08,2,NaN,금융 비트코인


In [3]:
# 학습된 모델 불러오기
model_path = "../models/ko_simcse_model"
model = AutoModel.from_pretrained(model_path).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [4]:
# category_keywords: 각 카테고리의 검색 키워드에서 "/"로 구분된 키워드를 공백(" ")으로 대체하여 리스트로 저장 (임베딩을 통한 카테고리 분류에 사용)
category_keywords = [" ".join(i.split("/")) for i in category_file[category_file['matching_type']==2]["검색키워드"]]

# tokenizer: 각 카테고리의 키워드를 토크나이저로 토큰화, 패딩과 트렁케이션을 적용하여 텐서로 변환
inputs = tokenizer(category_keywords, padding=True, truncation=True, return_tensors="pt")

# inputs: 변환된 텐서를 CUDA 장치로 이동 (GPU 사용을 위해)
inputs = {key: tensor.to('cuda') for key, tensor in inputs.items()}

# label_embedding: 모델을 사용해 입력된 키워드의 임베딩을 생성 (카테고리별로 임베딩 생성)
# return_dict=False는 모델이 dict 대신 튜플 형태로 결과를 반환하도록 설정
label_embedding, _ = model(**inputs, return_dict=False)

# label_embedding.size(): 생성된 임베딩의 크기를 출력 (임베딩 텐서의 크기 확인)
label_embedding.size()


c:\Users\csu52\anaconda3\envs\csu\lib\site-packages\transformers\models\roberta\modeling_roberta.py:370: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


torch.Size([107, 28, 768])

In [5]:
# 테스트 데이터 불러오기
df = pd.read_csv("../data/video_sample_test.csv", encoding="utf-8-sig", index_col=0)
df.head() 

,video_id,text,true,predict
0,q5g1V6B7aiE,불티나게 팔린 어묵탕 추천순위 TOP10 #대림선 #고래사어묵 #cj제일제당 #범...,방송 광고/이벤트,NaN
1,BY6uCMhkZFo,Cyber Space 1-5 Dropaholic ( Alpha Version ) S...,해외영상,NaN
2,yoeTDrUB-_o,[ 5 WORDS COMPILATION Learn 5 Korean Words Phr...,해외영상,NaN
3,2CpgC1K7jNI,대포죽순이요 푸바오 핸펀몰카사건 푸바오 이야기 FUBAO PANDA CUTE ANI...,라이프스타일 일상/vlog,NaN
4,KiRMy2Bu5SU,嫁に海老を食べさせられた甲殻類アレルギーの俺 → 発作を起こした俺がその場で倒れて意識不明に...,해외영상,NaN


In [6]:
### 거리계산 함수
def cal_score(a, b):
    # 입력 텐서가 1차원이면 2차원으로 변환
    if len(a.shape) == 1: a = a.unsqueeze(0)
    if len(b.shape) == 1: b = b.unsqueeze(0)

    # 텐서의 L2 노름을 사용해 정규화
    a_norm = a / a.norm(dim=1)[:, None]
    b_norm = b / b.norm(dim=1)[:, None]

    # 정규화된 텐서 간의 내적을 계산해 유사도를 측정하고 100을 곱해 반환
    return torch.mm(a_norm, b_norm.transpose(0, 1)) * 100 


### 카테고리에 따른 추가점수 함수
def add_score(youtube_category, sentences_similarity):
    # 유튜브 카테고리가 "Music"일 경우, 첫 번째 단어가 "음악"인 문장에 추가 점수 부여
    if youtube_category == "Music":
        for key in sentences_similarity:
            first_word = key.split()[0]
            if first_word == "음악":
                sentences_similarity[key] += 5

    # 유튜브 카테고리가 "Gaming"일 경우, 첫 번째 단어가 "게임"인 문장에 추가 점수 부여
    elif youtube_category == "Gaming":
        for key in sentences_similarity:
            first_word = key.split()[0]
            if (first_word == "게임"):
                sentences_similarity[key] += 5

    # 유튜브 카테고리가 "Sports"일 경우, 첫 번째 단어가 "스포츠"인 문장에 추가 점수 부여
    elif youtube_category == "Sports":
        for key in sentences_similarity:
            first_word = key.split()[0]
            if first_word == "스포츠":
                sentences_similarity[key] += 5

    # 유튜브 카테고리가 "News & Politics"일 경우, 첫 번째 단어가 "뉴스"인 문장에 추가 점수 부여
    elif youtube_category == "News & Politics":
        for key in sentences_similarity:
            first_word = key.split()[0]
            if first_word == "뉴스":
                sentences_similarity[key] += 5

    # 유튜브 카테고리가 "Travel & Events"일 경우, 첫 번째 단어가 "여행"인 문장에 추가 점수 부여
    elif youtube_category == "Travel & Events":
        for key in sentences_similarity:
            first_word = key.split()[0]
            if first_word == "여행":
                sentences_similarity[key] += 5
                
    # 다른 카테고리일 경우, 첫 번째 단어가 "게임"인 문장에 감점
    else:
        for key in sentences_similarity:
            first_word = key.split()[0]
            if first_word == "게임":
                sentences_similarity[key] -= 3

    return sentences_similarity


### 한글 여부 판독기 함수
def calculate_korean_ratio(text, ratio=.2):
    # E-mail 제거 패턴
    pattern = '([a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)'
    text = re.sub(pattern=pattern, repl='', string=text)

    # URL 제거 패턴
    pattern = '(http|ftp|https)://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    text = re.sub(pattern=pattern, repl=' ', string=text)

    # 특수문자 제거 후 공백 기준으로 단어 나눔
    clean_text = re.sub(r"[^\w\s]", " ", text)
    words = clean_text.split()

    # 전체 단어 수 및 한글 단어 수 계산
    total_words = len(words)
    korean_words = 0
    for word in words:
        for char in word:
            if '\uAC00' <= char <= '\uD7A3':  # 한글 유니코드 범위
                korean_words += 1

    try:
        # 한글 비율 계산 후 지정한 비율 이상일 경우 True 반환
        korean_ratio = korean_words / total_words
        if korean_ratio >= ratio:
            return True
        else:
            return False
    except ZeroDivisionError as e:
        # 단어가 없을 경우 False 반환
        return False
    

In [7]:
### 텍스트 분류 함수
def classification(text, etc_score=39):
    # 텍스트에서 유튜브 카테고리와 실제 텍스트를 분리
    youtube_category = text.split("[SEP]")[1].strip()  # [SEP] 기준으로 카테고리를 분리하여 저장
    text = text.split("[SEP]")[0].strip()  # 텍스트 부분 저장
    
    ### 외국채널 여부 확인
    if not calculate_korean_ratio(text, ratio=.1):
        return "해외영상", 9999, 0

    ### 검색키워드 일치 기반 분류
    content = set(text.split())  # 텍스트를 단어로 나누어 집합으로 저장
    for row in category_file[category_file['matching_type']==1].iterrows():
        category = row[1].category  # 카테고리 이름
        cluster = row[1].cluster  # 클러스터 번호
        keywords = set(row[1]["검색키워드"].split("/"))  # 검색 키워드를 집합으로 저장
        
        if len(content & keywords) > 0:  # 키워드와 텍스트의 교집합 확인
            return category, cluster, 0  # 일치하는 경우 해당 카테고리와 클러스터 반환
    ########################
    
    ### 텍스트 임베딩 기반 분류
    try:
        # 입력 텍스트를 토크나이저로 변환하고, CUDA 디바이스로 이동
        inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
        inputs = {key: tensor.to("cuda") for key, tensor in inputs.items()}

        # 모델을 사용해 임베딩 생성
        embedding = model(**inputs, return_dict=False)
    except Exception as e:
        # 임베딩 생성 실패 시 "기타" 카테고리로 분류
        print(text, e)
        return "기타", 0, 0
    
    # 임베딩 간 유사도 계산
    sentences_similarity = dict()
    for i, ce in enumerate(label_embedding):
        # 각 카테고리와의 유사도 계산 후 저장
        sentences_similarity[category_file[category_file['matching_type']==2]['category'].iloc[i]] = float(cal_score(embedding[0][0][0], ce[0])[0][0].item())

    # 유사도에 카테고리별 추가 점수 적용
    sentences_similarity = add_score(youtube_category, sentences_similarity)

    # 가장 큰 유사도 값 추출
    max_value = max(sentences_similarity.values())

    # 유사도가 기준점(etc_score)보다 낮을 경우
    if max_value < etc_score:
        content = text.split()
        # 검색 키워드를 사용해 카테고리와 클러스터 찾기
        for row in category_file[category_file['matching_type']==2].iterrows():
            keywords = set(row[1]['검색키워드'].split("/"))  # 검색 키워드를 집합으로 저장
            category = row[1]['category']  # 카테고리 이름
            category_cluster = row[1]['cluster']  # 클러스터 번호
            
            if len(set(content) & set(keywords)) > 0:  # 키워드와 텍스트의 교집합 확인
                return category, category_cluster, 0  # 일치하는 경우 해당 카테고리와 클러스터 반환
        
        # 해당하는 카테고리가 없으면 "기타"로 분류
        category = "기타"
        category_cluster = 0
    else:
        # 유사도가 높은 카테고리와 클러스터 반환
        category = [key for key, value in sentences_similarity.items() if value == max_value][0]
        category_cluster = category_file[category_file["category"]==category]["cluster"].iloc[0]
    
    return category, category_cluster, max_value  # 최종 카테고리, 클러스터, 유사도 반환


In [8]:
for row in tqdm(df.iterrows(), total=len(df)):
    text = row[1].text
    df.at[row[0], "predict"] = classification(text)[0]
df

100%|██████████| 30/30 [00:01<00:00, 19.68it/s]


,video_id,text,true,predict
0,q5g1V6B7aiE,불티나게 팔린 어묵탕 추천순위 TOP10 #대림선 #고래사어묵 #cj제일제당 #범...,방송 광고/이벤트,방송 광고/이벤트
1,BY6uCMhkZFo,Cyber Space 1-5 Dropaholic ( Alpha Version ) S...,해외영상,해외영상
2,yoeTDrUB-_o,[ 5 WORDS COMPILATION Learn 5 Korean Words Phr...,해외영상,해외영상
3,2CpgC1K7jNI,대포죽순이요 푸바오 핸펀몰카사건 푸바오 이야기 FUBAO PANDA CUTE ANI...,라이프스타일 일상/vlog,방송 광고/이벤트
4,KiRMy2Bu5SU,嫁に海老を食べさせられた甲殻類アレルギーの俺 → 発作を起こした俺がその場で倒れて意識不明に...,해외영상,해외영상
5,ya-XAGGBSpc,않은 직원들은 서운함 글로벌금융판매 보험회사지점장 보험 보험설계사 보험회사 김준호지...,금융 주식,뉴스 정치
6,PB8qtV2C884,许凯 ： 暗恋10年没表白 却用1000场吻戏 向谭松韵表达深沉的爱 101000 5 10...,해외영상,해외영상
7,SxoiIj5leQU,새티스팩토리 1.0ver 자동화 크래프팅 # 26 원형바닥 및 9티어재료만들기 군단...,게임 게임,게임 게임
8,9m7hBp5eSl0,コサキンが語る欽ちゃん 堂本光一さん 田中邦衛さん 大滝秀治さん kirinuki [SE...,해외영상,해외영상
9,RwBuOG03pB4,solo homeowner & everything wrong post renovat...,라이프스타일 일상/vlog,해외영상


In [10]:
# 정확도 계산
accuracy = accuracy_score(df['true'], df['predict'])

# F1 점수 계산 (다중 클래스일 경우 average='weighted' 사용)
f1 = f1_score(df['true'], df['predict'], average='weighted')

# 결과 출력
print(f'Accuracy: {accuracy:.2f}')
print(f'F1 Score: {f1:.2f}')

Accuracy: 0.77
F1 Score: 0.74
